# 基于MindSpore的GPT2文本摘要

该实验可进行在线体验，在线体验链接（https://pangu.huaweicloud.com/gallery/asset-detail.html?id=13ee0843-9209-4b5e-b99f-9ac05d1e0ed6
）

## 环境配置
>
>此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码
>

1. 配置python3.9环境


In [2]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

***注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0***

<div align=center><img src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG"></div>

2. 安装mindspore2.2.12，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)
3. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)

In [3]:
%%capture captured_output
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.11-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tokenizers==0.15.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
!wget https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/mindnlp_install/mindnlp-0.3.1-py3-none-any.whl
!pip install mindnlp

***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

### 数据集加载与处理

1. 数据集加载

    本次实验使用的是nlpcc2017摘要数据，内容为新闻正文及其摘要，总计50000个样本。

In [1]:
from mindnlp.utils import http_get

# download dataset
url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

  from .autonotebook import tqdm as notebook_tqdm

Building prefix dict from the default dictionary ...

Loading model from cache /tmp/jieba.cache

Loading model cost 0.761 seconds.

Prefix dict has been built successfully.


In [2]:
from mindspore.dataset import TextFileDataset

# load dataset
dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

**本案例默认在GPU P100上运行，因中文文本，tokenizer使用的是bert tokenizer而非gpt tokenizer等原因，全量数据训练1个epoch的时间约为80分钟。**

**为节约时间，我们选取了数据集中很小的一个子集（500条数据）来演示gpt2的微调和推理全流程，但由于数据量不足，会导致模型效果较差。**

In [3]:
# split into training and testing dataset
mini_dataset, _ = dataset.split([0.01, 0.99], randomize=False)
train_dataset, test_dataset = mini_dataset.split([0.9, 0.1], randomize=False)

2. 数据预处理

    原始数据格式：
    ```text
    article: [CLS] article_context [SEP]
    summary: [CLS] summary_context [SEP]
    ```
    预处理后的数据格式：

    ```text
    [CLS] article_context [SEP] summary_context [SEP]
    ```

In [4]:
import json
import numpy as np

# preprocess dataset
def process_dataset(dataset, tokenizer, batch_size=4, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        # tokenization
        # pad to max_seq_length, only truncate the article
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    # change column names to input_ids and labels for the following training
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

因GPT2无中文的tokenizer，我们使用BertTokenizer替代。

In [5]:
from mindnlp.transformers import BertTokenizer

# We use BertTokenizer for tokenizing chinese context.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
len(tokenizer)

21128

In [6]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=1)

In [7]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[1, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0]]),
 Tensor(shape=[1, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0]])]

### 模型构建

1. 构建GPT2ForSummarization模型，注意***shift right***的操作。

In [8]:
from mindspore import ops
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def construct(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().construct(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = ops.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return loss

2. 动态学习率

In [9]:
from mindspore import ops
from mindspore.nn.learning_rate_schedule import LearningRateSchedule

class LinearWithWarmUp(LearningRateSchedule):
    """
    Warmup-decay learning rate.
    """
    def __init__(self, learning_rate, num_warmup_steps, num_training_steps):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps

    def construct(self, global_step):
        if global_step < self.num_warmup_steps:
            return global_step / float(max(1, self.num_warmup_steps)) * self.learning_rate
        return ops.maximum(
            0.0, (self.num_training_steps - global_step) / (max(1, self.num_training_steps - self.num_warmup_steps))
        ) * self.learning_rate

### 模型训练

In [10]:
num_epochs = 1
warmup_steps = 100
learning_rate = 1.5e-4

num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [11]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2ForSummarization(config)

lr_scheduler = LinearWithWarmUp(learning_rate=learning_rate, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)

In [12]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [14]:
from mindnlp._legacy.engine import Trainer
from mindnlp._legacy.engine.callbacks import CheckpointCallback

ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt2_summarization',
                                epochs=1, keep_checkpoint_max=2)

trainer = Trainer(network=model, train_dataset=train_dataset,
                  epochs=num_epochs, optimizer=optimizer, callbacks=ckpoint_cb)
trainer.set_amp(level='O1')  # 开启混合精度

Trainer will use 'StaticLossScaler' with `scale_value=2 ** 10` when `loss_scaler` is None.


In [ ]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/450 [00:00<?, ?it/s][WARNING] KERNEL(18952,7f8cb7fff700,python):2024-05-26-02:43:45.191.987 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel.cc:40] CheckDeviceSm] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6

Epoch 0: 100%|██████████| 450/450 [04:18<00:00,  1.74it/s, loss=7.1492634]


Checkpoint: 'gpt2_summarization_epoch_0.ckpt' has been saved in epoch: 0.


In [15]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [16]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [17]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101,  126, 3299, 8110, 3189, 3241, 8024, 3633,  966, 2844, 1894,

        5688,  511, 3727,  704, 2356,  704, 2552, 1278, 7368, 6117, 3890,

        7599, 3969, 1048, 4554, 4906, 2844, 1894, 2207, 6145, 8024, 3633,

        1762, 3146, 4415, 2843, 3131, 2642, 5442, 4500, 4638, 4289, 1501,

        8024, 1762, 3690, 3187, 7344, 1906, 4638, 2658, 1105,  678, 8024,

        6158,  671, 4511, 2094,  697, 3613, 3666, 2802,  511,  113, 1290,

        1555, 2845,  126, 3299, 8115, 3189,  100, 4276, 3295, 2845, 6887,

         114,  126, 3299, 8130, 3189,  677, 1286, 8024, 1290, 1555, 2845,

        6381, 5442,  794, 1062, 2128, 3727, 1378, 6356, 3175,  749, 6237,

        1168, 8024, 2802,  782, 4511, 2094, 1426, 3378, 2347, 6158, 6121,

        3124, 2872, 4522,  511, 2945, 6356, 3175,  792, 5305, 8024,  126,

        3299, 8110, 3189, 2496, 3241, 8024, 1728, 1426, 3378, 5326, 3678,

        1728, 2843, 3131, 3187, 3126, 3647,  767, 8024, 6628, 1168, 1278,

        7368, 4638, 1426

In [18]:
model = GPT2LMHeadModel.from_pretrained('./checkpoint/gpt2_summarization_epoch_0.ckpt', config=config)

由于训练数据量少，epochs数少且tokenizer并未使用gpt tokenizer等因素，模型推理效果会较差。

In [19]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 1:
        break

/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1402: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )

  warnings.warn(

[WARNING] KERNEL(24053,7f01dd358700,python):2024-05-26-02:50:12.325.792 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel.cc:40] CheckDeviceSm] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6


[CLS] 5 月 12 日 晚 ， 正 值 护 士 节 。 汉 中 市 中 心 医 院 血 液 风 湿 免 疫 科 护 士 小 袁 ， 正 在 整 理 抢 救 患 者 用 的 物 品 ， 在 毫 无 防 备 的 情 况 下 ， 被 一 男 子 两 次 殴 打 。 ( 华 商 报 5 月 15 日 [UNK] 版 曾 报 道 ) 5 月 22 日 上 午 ， 华 商 报 记 者 从 公 安 汉 台 警 方 了 解 到 ， 打 人 男 子 吴 某 已 被 行 政 拘 留 。 据 警 方 介 绍 ， 5 月 12 日 当 晚 ， 因 吴 某 继 母 因 抢 救 无 效 死 亡 ， 赶 到 医 院 的 吴 某 情 绪 激 动 ， 将 护 士 站 护 士 小 袁 打 伤 ， 导 致 小 袁 头 部 外 伤 ， 全 身 多 处 软 组 织 挫 伤 。 医 院 报 警 后 ， 吴 某 逃 离 现 场 。 事 件 发 生 后 ， 民 警 第 一 时 间 展 开 调 查 ， 锁 定 嫌 疑 人 吴 某 。 随 后 ， 吴 某 被 警 方 传 唤 ， 对 违 法 事 实 供 认 不 讳 。 目 前 ， 吴 某 因 涉 嫌 寻 衅 滋 事 ， 被 警 方 行 政 拘 留 15 日 。 华 商 报 记 者 从 汉 中 市 中 心 医 院 了 解 到 ， 小 袁 已 出 院 ， 回 到 自 己 的 工 作 岗 位 。 华 商 记 者 < [UNK] > 王 亮 编 辑 ： 华 商 报 供 稿 [SEP] ， 日 ， ， 。 。 ， 人 ， [UNK] ， 道 ， 中 ， 的 ， 年 ， 出 ， 时 ，. ， - ， 天 ， 月 ， 场 ， 温 ， 后 。 日 日 人 。 [UNK] 。 道 。 中 。 的 。 人
